In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2-5-dataset/2_die_5_move/input4.json
/kaggle/input/2-5-dataset/2_die_5_move/input1.json
/kaggle/input/2-5-dataset/2_die_5_move/input3.json
/kaggle/input/2-5-dataset/2_die_5_move/input2.json


In [2]:
import json
import numpy as np
import os
import random
from collections import deque
from gym import Env
from gym.spaces import Discrete, Box
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
def load_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        # print(data)
    board_start = [list(map(int, row)) for row in data['board']['start']]
    board_goal = [list(map(int, row)) for row in data['board']['goal']]
    patterns = [
        {
            'id': pattern['p'],
            'width': pattern['width'],
            'height': pattern['height'],
            'cells': [list(map(int, row)) for row in pattern['cells']]
        }
        for pattern in data['general']['patterns']
    ]
    return board_start, board_goal, patterns

In [ ]:
class BoardEnvWithTwoDies(Env):
    def __init__(self, board, goal, dies, max_steps=5):
        """
        Môi trường RL cho bài toán 2 khuôn.

        Args:
            board: Bảng bắt đầu (numpy array hoặc list of lists).
            goal: Bảng mục tiêu (numpy array hoặc list of lists).
            dies: Danh sách các khuôn (list of dicts).
            max_steps: Số bước tối đa.
        """
        super(BoardEnvWithTwoDies, self).__init__()
        self.board = np.array(board)
        self.goal = np.array(goal)
        self.dies = dies  # Danh sách các khuôn
        self.num_dies = len(dies)  # Số khuôn
        self.width, self.height = self.board.shape
        self.max_steps = max_steps
        self.current_step = 0

        # Không gian hành động: (chọn khuôn, x, y, hướng)
        self.action_space = Discrete(self.num_dies * self.width * self.height * 4)
        # Không gian quan sát: trạng thái của bảng
        self.observation_space = Box(low=0, high=3, shape=(self.width, self.height), dtype=int)

    def reset(self):
        """Đặt lại môi trường về trạng thái ban đầu."""
        self.state = self.board.copy()
        self.current_step = 0
        return self.state

    def step(self, action):
        """Thực hiện một hành động và trả về trạng thái tiếp theo."""
        die_index, x, y, direction = self.decode_action(action)
        selected_die = self.dies[die_index]

        # Áp dụng khuôn và dịch chuyển
        new_board = self.state.copy()
        new_board = self.apply_pattern(new_board, selected_die, (x, y), direction)

        # Tính phần thưởng
        reward = -1
        done = False
        if np.array_equal(new_board, self.goal):
            reward = 100
            done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            done = True

        self.state = new_board
        return new_board, reward, done, {}

    def decode_action(self, action):
        """Giải mã hành động thành (khuôn, x, y, hướng)."""
        num_cells = self.width * self.height * 4  # Số ô và hướng
        die_index = action // num_cells
        remainder = action % num_cells

        x = remainder // (self.height * 4)
        remainder = remainder % (self.height * 4)

        y = remainder // 4
        direction = remainder % 4

        return die_index, x, y, direction

    def apply_pattern(self, board, pattern, position, direction):
        """Áp dụng khuôn lên bảng và xử lý dịch chuyển."""
        x, y = position
        pattern_width, pattern_height = pattern['width'], pattern['height']
        cells = pattern['cells']
        board_width, board_height = board.shape

        # Áp dụng khuôn
        lifted = []
        for i in range(pattern_height):
            for j in range(pattern_width):
                bx, by = x + j, y + i
                if 0 <= bx < board_width and 0 <= by < board_height:
                    if cells[i][j] == 1:
                        lifted.append(board[by][bx])
                        board[by][bx] = -1  # Đánh dấu ô rỗng

        # Dịch chuyển các ô bị nâng lên
        board = self.shift_board(board, direction, lifted)
        return board

    def shift_board(self, board, direction, lifted):
        """Dịch chuyển các ô được nâng lên theo hướng."""
        board_width, board_height = board.shape
        if direction == 0:  # Top
            for col in range(board_width):
                column = [board[row][col] for row in range(board_height) if board[row][col] != -1]
                column = [-1] * (board_height - len(column)) + column
                for row in range(board_height):
                    board[row][col] = column[row]

        elif direction == 1:  # Bottom
            for col in range(board_width):
                column = [board[row][col] for row in range(board_height) if board[row][col] != -1]
                column = column + [-1] * (board_height - len(column))
                for row in range(board_height):
                    board[row][col] = column[row]

        elif direction == 2:  # Left
            for row in range(board_height):
                row_data = [board[row][col] for col in range(board_width) if board[row][col] != -1]
                row_data = [-1] * (board_width - len(row_data)) + row_data
                for col in range(board_width):
                    board[row][col] = row_data[col]

        elif direction == 3:  # Right
            for row in range(board_height):
                row_data = [board[row][col] for col in range(board_width) if board[row][col] != -1]
                row_data = row_data + [-1] * (board_width - len(row_data))
                for col in range(board_width):
                    board[row][col] = row_data[col]

        empty_cells = [(r, c) for r in range(board_height) for c in range(board_width) if board[r][c] == -1]
        for idx, (r, c) in enumerate(empty_cells):
            board[r][c] = lifted[idx] if idx < len(lifted) else -1

        return board

In [5]:


class DQNAgent:
    def __init__(self, state_shape, action_size):
        self.state_shape = state_shape
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Flatten(input_shape=self.state_shape),
            Dense(128, activation='relu'),
            Dense(128, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state[np.newaxis])
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state[np.newaxis])[0])
            target_f = self.model.predict(state[np.newaxis])
            target_f[0][action] = target
            self.model.fit(state[np.newaxis], target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [6]:
def train_dqn(env, agent, episodes, batch_size=64):
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        for _ in range(env.max_steps):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            if done:
                break
        print(f"Episode {episode+1}/{episodes} - Reward: {total_reward}")
        agent.replay(batch_size)


In [ ]:

board_start, board_goal, patterns = load_data("/kaggle/input/2-5-dataset/2_die_5_move/input2.json")

# Create environment
env = BoardEnvWithTwoDies(board_start, board_goal, patterns, max_steps=5)

agent = DQNAgent(env.observation_space.shape, env.action_space.n)

# #     # Train agent
train_dqn(env, agent, 500, batch_size=64)
    
# #     # Save model
agent.model.save("RL_procon_1_die_2_move.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Episode 1/500 - Reward: -5
Episode 2/500 - Reward: -5
Episode 3/500 - Reward: -5
Episode 4/500 - Reward: -5
Episode 5/500 - Reward: -5
Episode 6/500 - Reward: -5
Episode 7/500 - Reward: -5
Episode 8/500 - Reward: -5
Episode 9/500 - Reward: -5
Episode 10/500 - Reward: -5
Episode 11/500 - Reward: -5
Episode 12/500 - Reward: -5
Episode 13/500 - Reward: -5


I0000 00:00:1733851416.679929      68 service.cc:145] XLA service 0x7d3360005c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733851416.679984      68 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733851416.679988      68 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step


I0000 00:00:1733851417.196504      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━